# EDA on Streaming Subscription Dataset  
This notebook explores subscription data, analyzing customer retention, churn, revenue metrics, and cohort patterns to uncover actionable insights for improving business decisions.


## 1. Setup & Data Loading  
We start by importing the required libraries and loading the subscription dataset into a Pandas DataFrame.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("Subscription Cohort Analysis Data.csv")

In [ ]:
df

## 2. Data Overview  
We inspect the structure of the dataset, data types, and a quick statistical summary to understand the scale of data and potential missing values.
- **Key Observation:** There are 3069 rows, and `canceled_date` has missing values for active subscriptions.


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

Converting 2 columns into Datetime

In [ ]:
df['created_date'] = pd.to_datetime(df['created_date'])
df['canceled_date'] = pd.to_datetime(df['canceled_date'])

In [ ]:
df.info()

In [ ]:
# Converting "was_subscription_paid" feild into True False instead of Yes, No
df['was_subscription_paid'].sample(8)

In [ ]:
mapping = {"Yes":True, "No":False}
df['was_subscription_paid'] = df['was_subscription_paid'].map(mapping)

In [ ]:
df.head()

In [ ]:
df['subscription_interval'].value_counts()

In [ ]:
# In this the missing values indicate the number of active suscriptions
df['canceled_date'].isnull().sum()

## 3. Data Cleaning & Feature Engineering  
We create new columns to make the analysis easier:
- `subscription_duration_days` → Number of days a subscription lasted
- `is_active` → Binary column indicating whether the subscription is still active
- `signup_month` → Month of subscription start
- `subscription_number` → Number of times a customer has subscribed
- `days_since_last_cancellation` → Time since last cancellation

These features will help in churn analysis and cohort segmentation.


In [ ]:
df.columns

In [ ]:
df['subscription_duration_days'] = (df['canceled_date'].fillna(pd.Timestamp.today())  - df['created_date']).dt.days

In [ ]:
df['is_active'] = df['canceled_date'].isna()

In [ ]:
# Month / Year Singup
df['singup_month'] = df['created_date'].dt.to_period('M')

In [ ]:
df

In [ ]:
df.duplicated().sum()

In [ ]:
df['customer_id'].duplicated().sum()

In [ ]:
df.info()

 Investigate the Duplicates of Customer_id Before Dropping.
They could be re-subscribtions

In [ ]:
duplicates = df[df.duplicated(subset=['customer_id'], keep=False)]
duplicates.sort_values(by=['customer_id', 'created_date'])

this confirms that those "duplicates" are not duplicates in the bad-data sense, but actual churn + resubscribe events

customer_id 116060198 canceled on 2023-03-20, then signed up again on 2023-04-24.

customer_id 119436804 canceled on 2023-04-10, then signed up again on 2023-05-21.

In [ ]:
# Creating a new coulmn subscription_number  that shows the count of subscription to dsistinct from rescribers and single time subscribers.
df['subscription_number'] = df.groupby('customer_id').cumcount() + 1

Also adding another days column that shows how often people are return after canceling the subscription

In [ ]:
df['days_since_last_cancellation'] = (
    df.groupby('customer_id')['created_date'].diff().dt.days)


In [ ]:
df.head()

# EDA
In this section, we explore the subscription dataset to uncover patterns, trends, and anomalies.  
Our goal is to understand customer behavior, identify churn patterns, measure retention, and evaluate the overall health of the subscription business.  

The analysis is divided into the following key parts:
- **Churn & Retention Analysis** – Measure churn rate, early churn, and active subscriber base.
- **Revenue Metrics** – Calculate ARPU (Average Revenue per User) and overall revenue contribution.
- **Cohort Analysis** – Group customers by signup month to study retention over time.
- **Visualizations** – Plot trends and patterns to make the insights actionable.

In [ ]:
# Basic Dataset Understanding

In [ ]:
total_customers = df['customer_id'].nunique()
print(f"Total Unique customers count: {total_customers}")

total_subscriptions = len(df)
print(f"Total subscriptions: {total_subscriptions}")

repeat_customer = df[df['subscription_number'] > 1]['customer_id'].nunique()
print(f"Repeat Customers Count: {repeat_customer}")

pect_of_repeat = round((repeat_customer / total_subscriptions) * 100)
print(f"Percentage of Repeated Customers: {pect_of_repeat}% ")

In [ ]:
# Subscription Duration
df['subscription_duration_days'].describe()


In [ ]:
# % of subscriptions with duration  < 7 days

short_duration_count = (df['subscription_duration_days'] < 7).sum()
total_subscriptions = len(df)
print(f"Number of subscriptions less than 7 days: {short_duration_count}")

if total_subscriptions > 0:
  perct_short_days = (short_duration_count / total_subscriptions) * 100
else:
  perct_short_days = 0
print(f"Percentage of short-duration subscriptions: {perct_short_days:.2f}%")

In [ ]:
percentages = (df['was_subscription_paid'].value_counts(normalize=True) * 100).map('{:.0f}%'.format)
print(percentages)

In [ ]:
# singup of per month
df['singup_month'].value_counts().sort_index()

In [ ]:
# Cancellations per Month count of each month
df.dropna(subset=['canceled_date'])['canceled_date'].dt.to_period('M').value_counts().sort_index()

## 4. Churn & Retention Analysis  
We calculate key subscription health metrics:
- **Churn Rate:** X% of customers have churned (i.e., are inactive).
- **Early Churn:** Y% of churn happens within the first 7 days.
- **ARPU:** The average revenue per user is ₹Z.

**Insights:**
- A high early churn rate may indicate onboarding issues.
- ARPU gives a measure of customer value and can be used for forecasting.

In [ ]:
# Find when customers are dropping off.
churn_rate = 1 - df['is_active'].mean()
print(f"Churn Rate: {churn_rate * 100:.2f}%")

In [ ]:
# Churn by Subscription Number
churn_by_subscription = (
    1 - df.groupby('subscription_number')['is_active'].mean()) * 100
print(churn_by_subscription)

In [ ]:
# Distribution of Subscription Duration
df['subscription_duration_days'].hist(bins=30)

In [ ]:
# Check if most churn happens early (common in subscription businesses)
early_churn = (df['subscription_duration_days'] <= 7).mean()
print(f"\nEarly Churn (<=7 days): {early_churn * 100:.2f}%")

Reactivation Analysis

In [ ]:
# Study returning customers.
df[df['subscription_number'] > 1]['days_since_last_cancellation'].describe()

In [ ]:
# Gap Distribution
# Look at how quickly users resubscribe (short vs long gaps)

Revenue & Payment Analysis

In [ ]:
total_revenue = df[df['was_subscription_paid']]['subscription_cost'].sum()
print(total_revenue)

In [ ]:
# Average Revenue Per User (ARPU)
arpu = total_revenue / total_customers
print(f"\nAverage Revenue Per User (ARPU): ${arpu:.2f}")

In [ ]:
# Lost Revenue (Unpaid)
unpaid_loss = df[~df['was_subscription_paid']]['subscription_cost'].sum()
print(unpaid_loss)

## 5. Cohort Analysis  
We perform cohort analysis by signup month to measure retention across time.

**Insights:**
- Identify which months show higher retention.
- Spot trends in customer drop-off rates over time.

In [ ]:
# % of users from each signup month remain subscribed in later months.
# Filtering the first-time subscribers only:
first_subscriptions = df[df['subscription_number'] == 1].copy()


In [ ]:
first_subscriptions.shape

In [ ]:
# Create a column for cohort month:
first_subscriptions['cohort_month'] = first_subscriptions['created_date'].dt.to_period('M')

In [ ]:
# Create a column for active month (month of activity):
first_subscriptions['active_month'] = first_subscriptions['created_date'] + pd.to_timedelta(first_subscriptions['subscription_duration_days'], unit='D')
first_subscriptions['active_month'] = first_subscriptions['active_month'].dt.to_period('M')

In [ ]:
# Calculate cohort index (how many months after signup they are still active):
first_subscriptions['cohort_index'] = (
    (first_subscriptions['active_month'] - first_subscriptions['cohort_month']).apply(lambda x: x.n)
)


In [ ]:
# Building a retention table:
cohort_data = (
    first_subscriptions.groupby(['cohort_month', 'cohort_index'])['customer_id'].nunique().unstack(fill_value = 0)
)

cohort_size = cohort_data.iloc[:,0]
retention = cohort_data.divide(cohort_size, axis=0) * 100
print(retention.round(2))


This gave us a matrix where:

Rows = signup month

Columns = months since signup

Values = % of customers still active

In [ ]:
crunch = 100 - retention
print(crunch.round(2))

Instead of retention, show what % of users churned in each month after signup.
This is just 100 - retention.

Which months had the most users coming back after churn

In [ ]:
reactivations = df[df['subscription_number'] > 1]
reactivation_cohort = (
    reactivations.groupby(['singup_month'])['customer_id']
    .nunique()
)
print(reactivation_cohort)

Paid vs Unpaid Cohort Retention

In [ ]:
paid_retention = (
    first_subscriptions[first_subscriptions['was_subscription_paid']]
    .groupby(['cohort_month', 'cohort_index'])['customer_id']
    .nunique()
    .unstack(fill_value=0)
)

paid_retention = paid_retention.divide(paid_retention.iloc[:, 0], axis=0) * 100
print(paid_retention.round(2))

In [ ]:
import plotly.express as px

status_counts = df['is_active'].value_counts().reset_index()
status_counts.columns = ['is_active', 'count']
status_counts['is_active'] = status_counts['is_active'].map({True: 'Active', False: 'Churned'})

fig = px.pie(
    status_counts,
    values='count',
    names='is_active',
    title='<b>Active vs. Churned Subscribers</b>',
    hole=0.4,
    color='is_active',
    color_discrete_map={'Active': 'mediumturquoise', 'Churned': 'gold'}
)
fig.update_traces(
    textinfo='percent+label',
    marker=dict(line=dict(color='#FFFFFF', width=2)),
    pull=[0.05, 0],
    hovertemplate="<b>%{label}</b><br>Subscribers: %{value}<br>Proportion: %{percent}"
)
fig.update_layout(
    title_x=0.5,
    legend_title_text='Subscription Status',
    annotations=[dict(text='Status', x=0.5, y=0.5, font_size=20, showarrow=False)]
)
fig.show()

In [ ]:
signup_trend = df.groupby('singup_month')['customer_id'].nunique().reset_index()
signup_trend.columns = ['signup_month', 'unique_customers']

signup_trend['signup_month'] = signup_trend['signup_month'].astype(str)

fig = px.bar(
    signup_trend,
    x='signup_month',
    y='unique_customers',
    title="<b>Customer Signups Over Time</b>",
    labels={'signup_month': 'Signup Month', 'unique_customers': 'Unique Customers'},
    text='unique_customers',
    color='unique_customers',
    color_continuous_scale='Tealgrn',
    template='plotly_white'
)
fig.update_xaxes(
    type='category',
    tickangle=-45,
    showline=True,
    linewidth=1,
    linecolor='black'
)
fig.update_yaxes(
    showline=True,
    linewidth=1,
    linecolor='black'
)
fig.update_traces(
    texttemplate='%{text}',
    textposition='outside',
    marker=dict(line=dict(color='black', width=1.2))
)
fig.update_layout(
    title_x=0.5,
    coloraxis_showscale=False,
    margin=dict(t=60, b=60, l=60, r=40),
    font=dict(size=14),
    bargap=0.2
)
fig.show()

In [ ]:
churn_by_sub = df.groupby('subscription_number')['is_active'].mean().reset_index()
churn_by_sub['churn_rate'] = 1 - churn_by_sub['is_active']
churn_by_sub['subscription_number'] = churn_by_sub['subscription_number'].astype(str)

fig = px.bar(
    churn_by_sub,
    x='subscription_number',
    y='churn_rate',
    title="<b>Churn Rate by Subscription Number</b>",
    labels={'churn_rate': 'Churn Rate', 'subscription_number': 'Subscription Number'},
    text=churn_by_sub['churn_rate'].map(lambda x: f"{x:.1%}"),
    color='churn_rate',
    color_continuous_scale='Tealgrn',
    template='plotly_white'
)
fig.update_xaxes(
    type='category',
    showline=True,
    linewidth=1,
    linecolor='black'
)
fig.update_yaxes(
    showline=True,
    linewidth=1,
    linecolor='black',
    tickformat=".0%"
)
fig.update_traces(
    textposition='outside',
    marker=dict(line=dict(color='black', width=1.2))
)
fig.update_layout(
    title_x=0.5,
    coloraxis_showscale=False,
    margin=dict(t=60, b=60, l=60, r=40),
    font=dict(size=14),
    bargap=0.2
)
fig.show()


In [ ]:
early_churn_df = pd.DataFrame({
    "Category": ["Early Churned (≤7 days)", "Others"],
    "Count": [
        df[df['subscription_duration_days'] <= 7].shape[0],
        df[df['subscription_duration_days'] > 7].shape[0]
    ]
})
fig = px.pie(
    early_churn_df,
    values='Count',
    names='Category',
    title="<b>Early Churn vs. Others</b>",
    hole=0.45,
    color='Category',
    color_discrete_map={
        "Early Churned (≤7 days)": "gold",
        "Others": "mediumturquoise"
    }
)
fig.update_traces(
    textinfo='percent+label',
    marker=dict(line=dict(color='white', width=2)),
    pull=[0.08, 0],
    hovertemplate="<b>%{label}</b><br>Subscribers: %{value}<br>Proportion: %{percent}"
)
fig.update_layout(
    title_x=0.5,
    legend_title_text='Subscriber Group',
    annotations=[dict(text='Churn', x=0.5, y=0.5, font_size=18, showarrow=False)],
    font=dict(size=14),
    margin=dict(t=60, b=20, l=40, r=40)
)
fig.show()


# Key Insights from the Analysis

Here’s what stood out from the subscription data after our EDA:

- **Customer Base:**  
  We have **2,877 unique customers** but a total of **3,069 subscriptions**, meaning some customers have resubscribed.  
  Only **6% of customers are repeat buyers** — this is relatively low for a subscription-based business.  
  Improving retention and incentivizing renewals could help grow this number.

- **Churn Rate is High:**  
  The **overall churn rate is ~65%**, which is not a good sign.  
  High churn means we are losing a large portion of our customer base and spending more effort (and money) on acquiring new customers instead of keeping existing ones.

- **Subscription Duration:**  
  About **10.9% of subscriptions lasted less than 7 days**, which is a strong signal of **early churn**.  
  This could mean customers are signing up but not finding enough value to continue.  
  Addressing onboarding and initial user experience could help reduce this early drop-off.

- **Payment Status:**  
  **96% of subscriptions were paid successfully** while **4% failed**.  
  This is a positive sign — payment collection does not seem to be a major issue.

- **Subscription Number vs Churn:**  
  Churn is **highest for first-time subscribers (67%)** but drops sharply for second subscriptions (36%) and is almost zero for third subscriptions.  
  This is encouraging — customers who give us a second chance are much more likely to stick around.  
  Focusing on getting churned users to resubscribe could significantly improve retention.

- **Revenue Metrics:**  
  The **ARPU (Average Revenue Per User) is $39.80**, which makes sense since every subscription is priced at $39.  
  This metric will become more useful if we introduce variable pricing tiers in the future.

---

**Overall Impression:**  
The data shows a healthy acquisition rate but **struggles with retention**. The first subscription cycle seems to be the biggest challenge, with a lot of customers leaving within days.  
Improving early user experience and running win-back campaigns for churned users could have a large impact on overall growth.
